This notebook generates the $\textbf{test samples}$ kernel density estimate (KDE) plots of the VR dataset for different class (Left or right) and uncertainty in figure 3A. The DR(eye)VE KDE plot was generated with similar method. 

### Preliminary step
At least one full training run needs to be completed before proceeding with generating the KDE plot.

### Code execution and intended outcome
The code utilizes the saved data split from one of 12 layers baseline ViT model training runs with full frames, to plot distribution of fixations across pixel coordinates using kernel density estimation. 

In [1]:
import os
import configparser
import sys

# Get the parent directory path
parent_directory = os.path.dirname(os.path.abspath('__file__'))

# Construct the path to the config.ini file in the parent directory
config_file_path = os.path.join(parent_directory, '..', 'config.ini')

# Load the configuration file
config = configparser.ConfigParser()
config.read(config_file_path)

parent_dir = os.path.abspath(os.path.join(os.path.abspath(''), '..'))
sys.path.append(parent_dir)

In [ ]:
import pickle
import seaborn as sns
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from training.utils import *

In [ ]:
# Access configuration parameters
model_type = config['model_config']['model_type'] # should be vit

dataset_type = config['data']['dataset_type']
train_data_cond = config['data']['train_data_cond'] 

batch_size = int(config['hyperparams'][f'batch_size_{dataset_type}'])

trained_model_path = f'data/{dataset_type}/pre_trained_models/{dataset_type}_{train_data_cond}_{model_type}/trained_LR_driving' 

run = 0 # run number to analyze
layer_num = False # number of layer (False = 12 layers)

saved_split_pkl = f'{trained_model_path}_{run}_{layer_num}_datasets.pkl'

In [ ]:
# load the data split from one of the traninig runs
with open(saved_split_pkl, 'rb') as fp:
    train_list, valid_list, test_list = pickle.load(fp)
train_loader, valid_loader, test_loader, train_data, valid_data, test_data = get_loaders(train_list, valid_list, test_list, batch_size, train_data_cond, test_data_cond)

In [ ]:
test_num = len(test_data.file_list) # number of trials for testing

all_trial_info_df = None # placeholder dataframe for all trial information

# extract the trial information for each trial in test dataset and concatenating them
for p in list(range(test_num)): 

    # extract test trial dictionary 
    # (available information: trial density, participant session and trial info, label, and fixation map)
    test_trial_info_dict = extract_trial_info(p, train_data, valid_data, test_data)

    # extract fixation coordinate
    viewed_mask = test_trial_info_dict['fixation_map'] > 0
    fixation_cood = np.where(np.squeeze(viewed_mask))
    fixation_x = fixation_cood[1].T
    fixation_x = fixation_x[:,np.newaxis]
    fixation_y = fixation_cood[0].T
    fixation_y = fixation_y[:,np.newaxis]

    # get fixation pixel intensity
    pixel_intensity = np.squeeze(test_trial_info_dict['fixation_map'])[fixation_cood]
    pixel_intensity = pixel_intensity[:,np.newaxis]
    # re-scaling fixation pixel intensity to range from 0 to 100
    pixel_intensity_scaled = MinMaxScaler().fit_transform(pixel_intensity)
    pixel_intensity_scaled = np.round(pixel_intensity_scaled*100).astype(int)

    # raise error if the dimension does not match
    if len(fixation_x) != len(fixation_y) != len(pixel_intensity): 
        raise Exception(f"Dimension missmatch for fixation at trial {p}") 

    # duplicate the same trial information for each pixel
    trial_info = np.empty((len(fixation_x), 4))
    trial_info[:,0] = p 
    trial_info[:,1] = test_trial_info_dict['trial_density']
    trial_info[:,2] = test_trial_info_dict['participant_trial_info']
    trial_info[:,3] = test_trial_info_dict['label']
    trial_info = np.hstack((trial_info, fixation_x, fixation_y, pixel_intensity, pixel_intensity_scaled))

    # construct dataframe for fixation coordinate related data
    trial_info_df = pd.DataFrame(trial_info, columns = ['trial','density','participant_info','Label','fixation_x','fixation_y','pixel_intensity','pixel_intensity_scaled'])
    trial_info_df['participant_info'] = test_trial_info_dict['participant_trial_info']
    trial_info_df['pixel_intensity_scaled'] = pixel_intensity_scaled

    # concatenate extract data for all trials
    all_trial_info_df = pd.concat([all_trial_info_df, trial_info_df])

all_trial_info_df['Label'] = all_trial_info_df.Label.replace({1: "Right", 0:"Left"}) # decode the label

In [ ]:
# split the dataset to high and low by uncertainty
all_trial_info_df['uncertainty'] = all_trial_info_df['density'].transform(lambda x: pd.qcut(x, 2, labels=['low','high']))

low_trials_info_df = all_trial_info_df[all_trial_info_df['uncertainty'] == "low"]
high_trials_info_df = all_trial_info_df[all_trial_info_df['uncertainty'] == "high"]

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(10, 5))

# define transparency for KDE plot
overall_alpha = 1
right_left_alpha = .5
# define color and font size
red = sns.color_palette('Pastel1', n_colors=3)[0]
gray = "#f7f7f7"
font_size = 16

# plot KDE plot based on uncertainty and class ("Left" or "Right")
for col_no, turn_class in enumerate(['Left','Right']):
    for row_no, uncertainty in enumerate(['High', 'Low']):

        # select uncertainty specific dataframe
        if uncertainty == 'High':
            trial_info = high_trials_info_df
        else:
            trial_info = low_trials_info_df

        # generate overall KDE
        ax_n = sns.kdeplot(data = trial_info[['fixation_x','fixation_y','pixel_intensity_scaled']], 
                  x="fixation_x", y="fixation_y", weights = 'pixel_intensity_scaled', fill=True,
                  ax=ax[row_no, col_no], color = gray, alpha=overall_alpha, levels=7)
        # generate class-specific KDE
        ax_n = sns.kdeplot(data = trial_info[['fixation_x','fixation_y','pixel_intensity_scaled', 'Label']],
                        x="fixation_x", y="fixation_y", weights = 'pixel_intensity_scaled', hue = 'Label', 
                        alpha = right_left_alpha, hue_order = [turn_class], palette = [red], fill=True,
                        legend = False, ax=ax[row_no, col_no])

        # set title, x and y axis 
        if row_no == 0:
            ax_n.set_title(f'{turn_class} Turn', fontsize = font_size)

        if col_no == 0 and row_no == 0:
            nl = '\n'
            ax_n.set_ylabel(f'{uncertainty} uncertainty', fontsize = font_size)
            ax_n.set_xticks([])
        elif col_no == 1 and row_no == 0:
            ax_n.set_xticks([])
            ax_n.set_yticks([])
            ax_n.set_ylabel('')
        else:
            ax_n.set_ylabel('')

        if col_no == 1 and row_no == 1:
            # ax_n.set_xlabel('X Pixel Coordinate', fontsize = font_size) # show x-axis uncomment this line and comment other 2
            ax_n.set_xticks([])
            ax_n.set_xlabel('')
            ax_n.set_yticks([])
        elif col_no == 0 and row_no == 1:
            ax_n.set_ylabel(f'{uncertainty} uncertainty', fontsize = font_size)
            # ax_n.set_xlabel('X Pixel Coordinate ', fontsize = font_size) # show x-axis uncomment this line and comment other 2
            ax_n.set_xticks([])
            ax_n.set_xlabel('')
        else:
            ax_n.set_xlabel('')

        ax_n.set_xlim(0, 224)
        ax_n.set_ylim(75, 175)

        plt.subplots_adjust(hspace=0.02, wspace=0.07)

# Add outer y axis
if len(fig.texts) != 0: 
    for txt in fig.texts:
        txt.set_visible(False)

fig.text(0.04, 0.5, 'Y Pixel Coordinate - VR', ha='center', va='center', rotation='vertical', fontsize = font_size)

In [ ]:
fig.savefig('vr_fixation_kde_plot.png', dpi = 1500, bbox_inches='tight') # save image